Import price data to train and test on

In [133]:
import pandas as pd

prices = pd.read_excel('Price.xlsx')
prices = prices.loc[prices['PriceArea'] == 'DK1']
prices = prices['PriceDKK'].values

# Temporarily limit the number of prices to 100
prices = prices[:100]

Define actions and states, and allowed actions in different states. Only depends on SOC

In [ ]:
import numpy as np

ACTIONS = ["Charge", "Discharge", "Do nothing"]
PRICE_STATE = ["Low", "Medium", "High"]
SOC_STATE = ["0%", "20%", "40%", "60%", "80%", "100%"]
ALLOWED_ACTIONS = {"0%":   ["Charge", "Do nothing"],
                   "20%":  ["Charge", "Discharge", "Do nothing"], 
                   "40%":  ["Charge", "Discharge", "Do nothing"],
                   "60%":  ["Charge", "Discharge", "Do nothing"],
                   "80%":  ["Charge", "Discharge", "Do nothing"],
                   "100%": ["Discharge", "Do nothing"],}

# Initialize the lookup-table
P_ssa = np.empty((len(PRICE_STATE), len(SOC_STATE), len(ACTIONS)), dtype=object)

Prepare and estimate probability of transitioning from one price state to another

In [ ]:
# Split prices into three states
low_price_threshold = np.percentile(prices, 33)
high_price_threshold = np.percentile(prices, 66)

# One-hot encoding of price states
price_states = np.array([prices < low_price_threshold,
                        (prices >= low_price_threshold) & (prices < high_price_threshold),
                         prices >= high_price_threshold])
price_states = price_states.astype(int)

# Compute probability of being in a price state given the current price state.
# Assumed independent of SOC state and action
# price_prop = P(s'|s) -> price_prop[i,j] = P(s'=j|s=i)
price_prop = (price_states @ np.roll(price_states,-1).T) / (price_states).sum(axis=1)


Compute the transitioning matrix for each given state (price and SOC) and action. Limit allowed actions

In [ ]:
for i, ps in enumerate(PRICE_STATE):
    for j, ss in enumerate(SOC_STATE):
        for k, ac in enumerate(ACTIONS):
            # Given that we are in state ps, ss and take action ac
            # what is the probability of transitioning to each state (ps', ss')
            if ac in ALLOWED_ACTIONS[ss]:
                if ac == "Charge":
                    c = 1
                elif ac == "Discharge":
                    c = -1
                else:
                    c = 0
                SOC_prop = np.zeros((len(SOC_STATE)))
                SOC_prop[j+c] = 1
                # state_trans = P(price'|price) * P(soc'|soc, action)
                state_trans = price_prop[i,:].reshape(-1,1) @ SOC_prop.reshape(1,-1)
                P_ssa[i,j,k] = state_trans
            else:
                P_ssa[i,j,k] = np.zeros((len(PRICE_STATE), len(SOC_STATE)))